## Create a synthetic dataset for evaluating RAG pipeline

In [1]:
import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

In [2]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv("OPEN_AI_KEY")

In [3]:
qdrant_client = QdrantClient(
    url="http://localhost:6333"
)

qdrant_client.create_collection(
    collection_name="Amazon-items-collection-02",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE))

True

In [4]:
df_items = pd.read_json('../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl', lines=True)

In [5]:
def preprocess_data(row):
    return f"{row['title']} {''.join(row['features'])}"

In [6]:
df_items["preprocessed_data"] = df_items.apply(preprocess_data, axis=1)

In [7]:
df_sample = df_items.sample(50, random_state=25)

In [8]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding

In [9]:
data_to_emded = df_sample["preprocessed_data"].tolist()
pointstructs = []
for i, data in enumerate(data_to_emded):
    embedding = get_embedding(data)
    pointstructs.append(
        PointStruct(
        id=i,
        vector=embedding,
        payload={"text": data}
    )
)

In [10]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-02",
    wait=True,
    points=pointstructs,
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

### Render a prompt to generate synthetic Eval reference dataset

In [11]:
import json

output_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "Suggested question.",
            },
            "chunk_ids": {
                "type": "array",
                "items": {
                    "type": "integer",
                    "description": "Index of the chunk that could be used to answer the question.",
                },
            },
            "answer_example": {
                "type": "string",
                "description": "Suggested answer grounded in the contexr.",
            },
            "reasoning": {
                "type": "string",
                "description": "Reasoning why the question could be answered with the chunks.",
            },
        },
    },
}


SYSTEM_PROMPT = f"""
I am building a RAG application. I have a collection of 50 chunks of text.
The RAG application will act as a shopping assistant that can answer questions about the stock of the products we have available.
I will provide all of the available products to you with indexes of each chunk.
I want you to come up with 30 questions to which the answers could be grounded in the chunk context.
As an output I need you to provide me the list of questions and the indexes of the chunks that could be used to answer them.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose the chunks to answer the questions.
Try to have a mix of questions that could use multipple chunks and questions that could use single chunk.
Also, include 5 questions that can't be answered with the available chunks.


{json.dumps(output_schema, indent=2)}


I need to be able to parse the json output.
"""

USER_PROMPT = f"""
Here is the list of chunks, each list element is a dictionary with id and text:
{[{"id": i, "text": data} for i, data in enumerate(data_to_emded)]}
"""

In [12]:
response = openai.chat.completions.create(
    model="gpt-4.1",  
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ]
)

### Clen the output

In [13]:
json_output = response.choices[0].message.content
json_output = json_output.replace("```json", "")
json_output = json_output.replace("```", "")
json_output = json_output.replace("// BEGIN UNANSWERABLE QUESTIONS SECTION (5)", "")
json_output = json.loads(json_output)

In [14]:
json_output

[{'question': 'Do you have any projectors with 4K resolution in stock?',
  'chunk_ids': [0, 42],
  'answer_example': 'Yes, we have two options: the Dangbei Mars Pro 4K Projector with HDR10 home theater capabilities and the CIBEST Full-Sealed Optical Engine Home Movie FHD Projector, which supports 4K output.',
  'reasoning': 'Chunk 0 provides details about the Dangbei Mars Pro 4K projector, and chunk 42 describes the CIBEST projector with 4K support.'},
 {'question': 'Which wireless earbuds do you have that are waterproof and suitable for sports?',
  'chunk_ids': [6, 7, 26, 46, 47],
  'answer_example': 'We have several options for waterproof wireless earbuds suitable for sports, including the WeurGhy Wireless Earbuds (IPX7), zivsivc Wireless Earbuds (IPX7), a set of 50H Playtime Over Ear Buds with Noise Cancelling (IP7), Bluetooth 5.3 headphones with IP7 rating, and the Aisizon Air Conduction Over Ear Sports Wireless Bluetooth Headphones.',
  'reasoning': 'These chunks describe wireless

### Upload the dataset to LangSmith

In [15]:
from langsmith import Client

client = Client(api_key=os.environ["LANGSMITH_API_KEY"])

dataset_name = "rag-evaluation-dataset"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Dataset for evaluating RAG pipeline"
)

In [16]:
for item in json_output:
    client.create_example(
        dataset_id=dataset.id,
        inputs={"question": item["question"]},
        outputs={
            "ground_truth": item["answer_example"],
            "context_ids": item["chunk_ids"],
            "contexts": [qdrant_client.retrieve(collection_name="Amazon-items-collection-02", ids=[id], with_payload=True)[0].payload["text"] for id in item["chunk_ids"]]
        }
    )